# Support Vector Machine - Generate Features
This notebook generates features from 'Evaluating Segmentation Error Without Ground Truth' by Kohlberger et al.

author = Caroline Magg <br>
date = 27 August 2020 <br> 

______________________________________
history: <br>
2020-27-08 Generate SVM Features <br>
2020-29-08 Generate SVM Features csv <br>
2020-03-09 Generate standardized SVM Features <br>
2020-11-09 Generate standardized SVM Features for 15-20 <br>

In [ ]:
import os
import sys
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import cv2
import scipy
import logging as log
import skimage.segmentation as segmentation
from scipy.spatial.distance import cdist

In [ ]:
log.basicConfig(format='%(levelname)s:%(message)s', level=log.INFO)

### Add dependencies

In [ ]:
# add KidsBrainProject main folder to paths
sys.path.append(os.path.abspath('../../'))
sys.path.append(os.path.abspath('../utils/'))

In [ ]:
# add path to data here
path_data = "../../Data/" 

data_brain = pd.read_csv("../../Data/data_Brain.csv")

# Data

In [ ]:
all_files = [ filename for filename in os.listdir("../../Data/") if filename.endswith( "csv" ) and 'test' in filename ]
all_files

In [ ]:
#df = pd.read_csv("../../Data/data_Brain.csv", delimiter=';')
df = pd.read_csv("../../Data/test_Brain.csv", delimiter=';')
df.tail()

In [ ]:
df.loc[130]

In [ ]:
df2 = df[df['PatientID']==15]
df2.tail()

In [ ]:
list_of_gt_mask = [cv2.imread(x, cv2.IMREAD_GRAYSCALE) for x in df2['Init']]
list_of_pred_mask = [cv2.imread(x, cv2.IMREAD_GRAYSCALE) for x in df2['Segm']]
list_of_img = [cv2.imread(x, cv2.IMREAD_GRAYSCALE) for x in df2['Img']]

In [ ]:
len(list_of_gt_mask), len(list_of_img), len(list_of_pred_mask)

In [ ]:
plt.imshow(list_of_img[2])

In [ ]:
idx = 2
segm = list_of_gt_mask[idx]
img = list_of_img[idx].copy()
plt.imshow(img, cmap='gray')
plt.show()
img[segm==0] = 0
plt.imshow(img, cmap='gray')

In [ ]:
np.unique(img)

In [ ]:
mask = img.copy()

# Features

In [ ]:
from SVMFeatures import SVMFeatures

In [ ]:
all_files = [ filename for filename in os.listdir("../../Data/") if filename.endswith( "csv" ) and 'test' in filename ]
all_files

In [ ]:
for file in all_files:
    df = pd.read_csv(os.path.join("../../Data/",file), delimiter=';')
    print(file)
    list_of_gt_mask = [cv2.imread(x, cv2.IMREAD_GRAYSCALE) for x in df['Init']]
    list_of_pred_mask = [cv2.imread(x, cv2.IMREAD_GRAYSCALE) for x in df['Segm']]
    list_of_img = [cv2.imread(x, cv2.IMREAD_GRAYSCALE) for x in df['Img']]
    indep_features, dep_features = SVMFeatures(list_of_img, list_of_gt_mask, list_of_pred_mask, 1).calculate(standardize=False)
    indep_features.to_csv(os.path.join("../../Data/features_standardized_test",file.replace('test','features')), index=False, sep=';')  
    dep_features.to_csv(os.path.join("../../Data/features_standardized_test",file.replace('test','error_metrics')), index=False, sep=';')
    #indep_features.to_csv(os.path.join("../../Data/features_standardized",file.replace('data','features')), index=False, sep=';')  
    #dep_features.to_csv(os.path.join("../../Data/features_standardized",file.replace('data','error_metrics')), index=False, sep=';')

In [ ]:
indep_features

In [ ]:
indep_features.columns

In [ ]:
len(list_of_gt_mask), len(list_of_img), len(list_of_pred_mask)

In [ ]:
test = pd.read_csv(os.path.join("../../Data/features_standardized_test",file.replace('test','features')), sep=';')
#test = pd.read_csv(os.path.join("../../Data/features_standardized",file.replace('data','features')), sep=';')

In [ ]:
np.shape(test)